# Important Libraries

In [15]:
import math
from graphviz import Digraph

# Methods

In [16]:
def trace(root):
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges
def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'})
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    dot.node(name=uid, label="{%s | data %.4f}" % (n.label, n.data), shape="record")
    if n._operation:
      dot.node(name = uid + n._operation, label = n._operation)
      dot.edge(uid + n._operation, uid)
  for n1, n2 in edges:
    dot.edge(str(id(n1)), str(id(n2)) + n2._operation)

  return dot

# Value Class

In [17]:
class Value:

    def __init__(self, data, label = "", _operation = "", _children = ()):
        self.data = data
        self.label = label
        self._operation = _operation
        self._prev = set(_children)
        

    def __repr__(self):
        _type = f"{type(self.data)}"
        result = f"Value(data = {self.data}, type = {_type})"
        return result

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        name_of_operation = " add "
        nodes = (self, other)
        operation = self.data + other.data
        result = Value(data = operation, _operation = name_of_operation, _children = nodes)
        return result

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        name_of_operation = " mul "
        nodes = (self, other)
        operation = self.data * other.data
        result = Value(data = operation, _operation = name_of_operation, _children = nodes)
        return result

    def __rmul__(self, other):
        return self + other

    def __neg__(self):
        name_of_operation = f"neg({self.data})"
        nodes = (self, )
        operation = -self.data
        result = Value(
            data = operation, 
            label = f"-{self.label}" if self.label else "", 
            _children = nodes, 
            _operation = name_of_operation)
        return result

    def __sub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        result = self + (-other)
        return result

    def __rsub__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return other + (-self)

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Power should be ineger or float"
        name_of_operation = " pow "
        nodes = (self, )
        operation = self.data ** other
        result = Value(
            data = operation, 
            label = f"x^{self.label}" if self.label else "", 
            _children = nodes, 
            _operation = name_of_operation)
        return result

    def __truediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        result = self * (other ** -1.0)
        return result

    def __rtruediv__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        return other * (self ** -1.0)

    def exp(self):
        x = self.data
        name_of_operation = " exp "
        nodes = (self, )
        operation = math.exp(x)
        result = Value(data = operation, _operation = name_of_operation, _children = nodes)
        return result

    def tanh(self):
        x = self.data
        name_of_operation = " tanh "
        nodes = (self, )
        operation = (math.exp(2.0 * x) - 1) / (math.exp(2.0 * x) + 1)
        result = Value(data = operation, _operation = name_of_operation, _children = nodes)
        return result